In [1]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-800bfd82-4891-4304-b947-51b0958ad07e;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 215ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/11 08:27:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/11 08:27:20 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

In [3]:
df = readCheckpoint(size='test').cache()

 8:27AM UTC on Mar 11, 2023 --- read and cache time: 10.411009073257446 seconds ---


In [4]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = true)



In [5]:
df.count()

124142075

In [6]:
#filter to 3 malicious hosts

def get_firsts(df):
    # this drops duplicates and gets the first appearance of each relationship entry
    window = Window.partitionBy("relationship").orderBy("timestamp")
    df_new = df.withColumn('relationship', concat(df.objectID, lit('->'),df.actorID) ) \
                .withColumn('rank', rank().over(window)) \
                .filter(col('rank') == 1) \
                .drop('rank') \
                .cache()
    df.unpersist()
    return df_new

In [7]:
df = get_firsts(df)

In [8]:
ev_mins = [row.event_minute for row in df.select("event_minute").distinct().collect()]
len(ev_mins)
# x is the number of hours
# y is the maximum number of events for any hour in a graph
# x_axis = 2*(x-1)+2
# y_axis = 2*(y-1)+2

60

In [52]:
<<<<<<< LOCAL CELL DELETED >>>>>>>
ev_mins = [row.event_minute for row in df.select("event_minute").distinct().collect()]
len(ev_mins)
# x = max events per min
# y = total minutes
# x_axis = 2*(x-1) + 2
# y_axis = 2*(y+2)

11

In [9]:
df.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour,malicious,relationship
0,01ab472d-19f4-4ff5-8af6-eb16da85ee45,2019-09-23 22:33:02.702,00008552-84cd-4238-a460-857b15abb0a6,d2c26be6-d5c4-48c0-b9ef-318a289020f1,FILE,READ,SysClient0006.systemia.com,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\OneDr...,...,\Device\HarddiskVolume1\ncr\kickoff\ncr_utilit...,None,None,None,None,33,23,22,0,00008552-84cd-4238-a460-857b15abb0a6->d2c26be6...
1,7647f142-05ff-4cd9-b3ec-fa01f57370cc,2019-09-23 14:50:57.390,0001b62c-d8e6-499b-a127-2f1b5aeae906,4d83fca5-7044-496d-9bc0-3c7fd7d248ac,PROCESS,OPEN,SysClient0621.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,...,None,None,None,None,None,50,23,14,0,0001b62c-d8e6-499b-a127-2f1b5aeae906->4d83fca5...
2,b736db13-c989-4518-92ab-d0eaeab37635,2019-09-23 14:50:57.390,0001b62c-d8e6-499b-a127-2f1b5aeae906,4d83fca5-7044-496d-9bc0-3c7fd7d248ac,PROCESS,OPEN,SysClient0621.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,...,None,None,None,None,None,50,23,14,0,0001b62c-d8e6-499b-a127-2f1b5aeae906->4d83fca5...
3,742308b3-591b-4a0f-bfc6-ffd7561e8f3a,2019-09-23 13:44:17.181,0001e11a-ac5f-4469-9a7d-4d66dfdc5a2a,98d06282-e731-4bd3-902b-999ead640dfa,FILE,MODIFY,SysClient0417.systemia.com,None,None,System,...,\Device\HarddiskVolume1\Program Files (x86)\Mi...,None,None,None,None,44,23,13,0,0001e11a-ac5f-4469-9a7d-4d66dfdc5a2a->98d06282...
4,1893d0c9-00e4-4178-8827-0ddd35e8bf29,2019-09-23 17:22:06.808,00020875-07dd-48f2-aafd-0ed054ad6897,b56a9404-9fcb-444b-82c9-2ff05b41e2c5,FLOW,INFO,SysClient0816.systemia.com,None,None,None,...,None,None,None,None,None,22,23,17,0,00020875-07dd-48f2-aafd-0ed054ad6897->b56a9404...


In [10]:
# create new dataframe to union and create duplicate rows
df_dup = df.withColumn('id', lit(None).cast(StringType()))

In [11]:
df_dup.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour,malicious,relationship
0,None,2019-09-23 22:33:02.702,00008552-84cd-4238-a460-857b15abb0a6,d2c26be6-d5c4-48c0-b9ef-318a289020f1,FILE,READ,SysClient0006.systemia.com,None,None,\Device\HarddiskVolume1\Windows\SysWOW64\OneDr...,...,\Device\HarddiskVolume1\ncr\kickoff\ncr_utilit...,None,None,None,None,33,23,22,0,00008552-84cd-4238-a460-857b15abb0a6->d2c26be6...
1,None,2019-09-23 14:50:57.390,0001b62c-d8e6-499b-a127-2f1b5aeae906,4d83fca5-7044-496d-9bc0-3c7fd7d248ac,PROCESS,OPEN,SysClient0621.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,...,None,None,None,None,None,50,23,14,0,0001b62c-d8e6-499b-a127-2f1b5aeae906->4d83fca5...
2,None,2019-09-23 14:50:57.390,0001b62c-d8e6-499b-a127-2f1b5aeae906,4d83fca5-7044-496d-9bc0-3c7fd7d248ac,PROCESS,OPEN,SysClient0621.systemia.com,None,None,\Device\HarddiskVolume1\Program Files (x86)\Go...,...,None,None,None,None,None,50,23,14,0,0001b62c-d8e6-499b-a127-2f1b5aeae906->4d83fca5...
3,None,2019-09-23 13:44:17.181,0001e11a-ac5f-4469-9a7d-4d66dfdc5a2a,98d06282-e731-4bd3-902b-999ead640dfa,FILE,MODIFY,SysClient0417.systemia.com,None,None,System,...,\Device\HarddiskVolume1\Program Files (x86)\Mi...,None,None,None,None,44,23,13,0,0001e11a-ac5f-4469-9a7d-4d66dfdc5a2a->98d06282...
4,None,2019-09-23 17:22:06.808,00020875-07dd-48f2-aafd-0ed054ad6897,b56a9404-9fcb-444b-82c9-2ff05b41e2c5,FLOW,INFO,SysClient0816.systemia.com,None,None,None,...,None,None,None,None,None,22,23,17,0,00020875-07dd-48f2-aafd-0ed054ad6897->b56a9404...


In [12]:
df_new = df.union(df_dup).sort("timestamp").cache()
df.unpersist()

DataFrame[id: string, timestamp: timestamp, objectID: string, actorID: string, object: string, action: string, hostname: string, user_name: string, privileges: string, image_path: string, parent_image_path: string, new_path: string, file_path: string, direction: string, logon_id: string, requesting_domain: string, requesting_user: string, event_minute: int, event_day: int, event_hour: int, malicious: int, relationship: string]

In [13]:
df_new.limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,...,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour,malicious,relationship
0,ec8b44f6-4bc6-49c6-bbb1-19486aa5a067,2019-09-23 03:22:03.283,05fd64e2-01e3-475d-867d-d2d28098ddcc,08f1cce5-dced-4f8c-a603-97df325538a8,FILE,WRITE,SysClient0213.systemia.com,None,None,\Device\HarddiskVolume1\lwabeat\\lwabeat.exe,...,None,None,None,None,None,22,23,3,0,05fd64e2-01e3-475d-867d-d2d28098ddcc->08f1cce5...
1,None,2019-09-23 03:22:03.283,05fd64e2-01e3-475d-867d-d2d28098ddcc,08f1cce5-dced-4f8c-a603-97df325538a8,FILE,WRITE,SysClient0213.systemia.com,None,None,\Device\HarddiskVolume1\lwabeat\\lwabeat.exe,...,None,None,None,None,None,22,23,3,0,05fd64e2-01e3-475d-867d-d2d28098ddcc->08f1cce5...
2,bc9af9c8-cbc3-4393-96b4-81b19a92465d,2019-09-23 03:22:03.389,05fd64e2-01e3-475d-867d-d2d28098ddcc,74ebc904-dbed-4c0b-b594-e72de07f59a1,FILE,WRITE,SysClient0213.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\svcho...,...,None,None,None,None,None,22,23,3,0,05fd64e2-01e3-475d-867d-d2d28098ddcc->74ebc904...
3,None,2019-09-23 03:22:03.389,05fd64e2-01e3-475d-867d-d2d28098ddcc,74ebc904-dbed-4c0b-b594-e72de07f59a1,FILE,WRITE,SysClient0213.systemia.com,None,None,\Device\HarddiskVolume1\Windows\System32\svcho...,...,None,None,None,None,None,22,23,3,0,05fd64e2-01e3-475d-867d-d2d28098ddcc->74ebc904...
4,3f3f44ad-d3bf-41eb-a473-de80552439b5,2019-09-23 03:22:03.692,ebdb3daa-800c-4eac-bbcf-e9d06bd733d3,08f1cce5-dced-4f8c-a603-97df325538a8,PROCESS,OPEN,SysClient0213.systemia.com,None,None,\Device\HarddiskVolume1\lwabeat\\lwabeat.exe,...,None,None,None,None,None,22,23,3,0,ebdb3daa-800c-4eac-bbcf-e9d06bd733d3->08f1cce5...


In [14]:
len(df_new.select('relationship').distinct().collect())

23/03/11 08:44:29 ERROR TaskSetManager: Total size of serialized results of 19 tasks (1055.6 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)


23/03/11 08:44:29 ERROR TaskSetManager: Total size of serialized results of 20 tasks (1111.2 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)


Py4JJavaError: An error occurred while calling o124.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 19 tasks (1055.6 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1182)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$han

In [ ]:
tot = df_new.count()

In [ ]:
df_ratios = df_new.groupBy("object") \
              .count() \
              .withColumnRenamed('count', 'cnt_per_group') \
              .withColumn('perc_of_count_total', (col('cnt_per_group') / tot) * 100 ) \
              .cache()

In [ ]:
df_ratios.limit(5).toPandas()

In [ ]:
df_new.limit(5).toPandas()

In [ ]:
df_new.limit(10).write.option("maxRecordsPerFile", 300000).option("header",True).mode("overwrite").csv(f"{s3_url_trusted}/prod/graph/test")